In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -jax
%aimport -jaxlib
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..","projects")))
print(sys.path)

In [ ]:
import jax
import jax.numpy as jnp
from stanza.data import Data, PyTreeData
from stanza.util import shape_tree

# dummy data
train_data = Data.from_pytree((jnp.ones((100,32,32,3)), jnp.ones((100, 10))))
test_data = Data.from_pytree((jnp.ones((100,32,32,3)), jnp.ones((100, 10))))
# cifar data
from stanza.data.cifar import cifar10
train_data, test_data = cifar10()
def map_fn(sample):
    x, y = sample
    x = x.astype(float) / 255
    y = jax.nn.one_hot(y, 10)
    return x, y
train_data = PyTreeData.from_data(train_data.map(map_fn))
test_data = PyTreeData.from_data(test_data.map(map_fn))

from stanza.util.ipython import display_image
display_image(train_data[0][0])

In [ ]:
import haiku as hk
from jax.random import PRNGKey
import jax
import optax
from stanza import Partial
from stanza.train.rich import RichReporter
from stanza.train import Trainer, SAMTrainer

def run(use_sam=False):
    def net(x):
        flat, _ = jax.flatten_util.ravel_pytree(x)
        mlp = hk.nets.MLP([128,10])
        y = mlp(flat).squeeze()
        y = jax.nn.softmax(y)
        return y
    net = hk.transform(net)
    params = net.init(PRNGKey(42), train_data[0][0])
    @jax.jit
    def loss_fn(params, _state, _rng_key, batch):
        x, y = batch
        y_pred = net.apply(params, None, x)
        loss = jnp.sum(jnp.square(y_pred - y)) 
        return _state, loss, {"loss": loss}
    
    with RichReporter() as r:
        trainer = SAMTrainer(batch_size=128) if use_sam else Trainer(batch_size=128)
        r = trainer.train(Partial(loss_fn), 
            train_data, PRNGKey(42), params, 
            epochs=10, hooks=[r])
run(use_sam=False)